In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

device = ('cuda' if torch.cuda.is_available() else 'cpu')
device

'cuda'

In [2]:
import pandas as pd

df = pd.read_parquet('data/preprocessed.parquet')
df.head()

,Avg Packet Size,Packet Length Mean,Bwd Packet Length Std,Packet Length Variance,Bwd Packet Length Max,Packet Length Max,Packet Length Std,Avg Fwd Segment Size,Fwd Packet Length Mean,Flow Bytes/s,...,Fwd Act Data Packets,Subflow Fwd Packets,Total Fwd Packets,Bwd Header Length,Flow IAT Mean,Flow IAT Min,Fwd IAT Min,Fwd IAT Mean,Idle Std,ClassLabel
0,9.00000,6.000000,0.000000,0.000000,0.0,6.0,0.000000,6.00000,6.00000,3.000000e+06,...,1,2,2,0,4.000000,4.0,4.0,4.0,0.0,0
1,9.00000,6.000000,0.000000,0.000000,0.0,6.0,0.000000,6.00000,6.00000,1.200000e+07,...,1,2,2,0,1.000000,1.0,1.0,1.0,0.0,0
2,9.00000,6.000000,0.000000,0.000000,0.0,6.0,0.000000,6.00000,6.00000,4.000000e+06,...,1,2,2,0,3.000000,3.0,3.0,3.0,0.0,0
3,9.00000,6.000000,0.000000,0.000000,0.0,6.0,0.000000,6.00000,6.00000,1.200000e+07,...,1,2,2,0,1.000000,1.0,1.0,1.0,0.0,0
4,81.63636,74.833336,119.511505,11562.151367,207.0,233.0,107.527443,69.14286,69.14286,1.474548e+06,...,5,7,7,104,60.900002,2.0,2.0,101.5,0.0,0


In [3]:
df.dtypes

Avg Packet Size             float32
Packet Length Mean          float32
Bwd Packet Length Std       float32
Packet Length Variance      float32
Bwd Packet Length Max       float64
Packet Length Max           float64
Packet Length Std           float32
Avg Fwd Segment Size        float32
Fwd Packet Length Mean      float32
Flow Bytes/s                float64
Avg Bwd Segment Size        float32
Bwd Packet Length Mean      float32
Fwd Packets/s               float32
Flow Packets/s              float64
Init Fwd Win Bytes            int32
Subflow Fwd Bytes             int32
Fwd Packets Length Total    float64
Fwd Act Data Packets          int32
Subflow Fwd Packets           int32
Total Fwd Packets             int32
Bwd Header Length             int64
Flow IAT Mean               float32
Flow IAT Min                float64
Fwd IAT Min                 float64
Fwd IAT Mean                float32
Idle Std                    float32
ClassLabel                    int64
dtype: object

In [4]:
from sklearn.model_selection import train_test_split

X = df.drop(columns=['ClassLabel'], axis=1)
y = df['ClassLabel']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=69)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((6069091, 26), (2601039, 26), (6069091,), (2601039,))

In [5]:
from sklearn.preprocessing import MinMaxScaler

mm_scaler = MinMaxScaler(feature_range=(0, 1))
X_train = mm_scaler.fit_transform(X_train)
X_test = mm_scaler.transform(X_test)

In [6]:
class IntrusionDetectorNet(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(IntrusionDetectorNet, self).__init__()
        self.fc1 = nn.Linear(input_dim, 128)  # Input to first hidden layer
        self.dropout1 = nn.Dropout(0.5)  # Dropout layer after first hidden layer
        self.fc2 = nn.Linear(128, 64)  # Second hidden layer
        self.dropout2 = nn.Dropout(0.5)  # Dropout layer after second hidden layer
        self.fc3 = nn.Linear(64, 32)  # Third hidden layer
        self.fc4 = nn.Linear(32, output_dim)  # Output layer

    def forward(self, x):
        x = torch.relu(self.fc1(x))  # Activation after first hidden layer
        x = self.dropout1(x)
        x = torch.relu(self.fc2(x))  # Activation after second hidden layer
        x = self.dropout2(x)
        x = torch.relu(self.fc3(x))  # Activation after third hidden layer
        x = torch.softmax(self.fc4(x), dim=1)  # Softmax for multi-class output
        return x


# model configuration
input_dim = X_train.shape[1]  # Replace with your input dimension
output_dim = 4  # Number of classes
model = IntrusionDetectorNet(input_dim, output_dim).to(device)
# model summary
print(model)

IntrusionDetectorNet(
  (fc1): Linear(in_features=26, out_features=128, bias=True)
  (dropout1): Dropout(p=0.5, inplace=False)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (dropout2): Dropout(p=0.5, inplace=False)
  (fc3): Linear(in_features=64, out_features=32, bias=True)
  (fc4): Linear(in_features=32, out_features=4, bias=True)
)


In [7]:
print(y_train.value_counts())

ClassLabel
0    5029865
3     864704
1     102210
2      72312
Name: count, dtype: int64


In [ ]:
import os

class_counts = torch.tensor([5029865, 102210, 72312, 864704], dtype=torch.float32).to(device)
class_weights = 1.0 / class_counts
class_weights = class_weights / class_weights.sum()

criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = optim.Adam(model.parameters(), lr=0.1)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train.to_numpy(), dtype=torch.long).to(device)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=10240, shuffle=True, num_workers=os.cpu_count()//2)

# Training loop
epochs = 10
for epoch in range(epochs):
    model.train()
    running_loss = 0.0

    for inputs, labels in train_loader:
        # Move batch data to the device
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()  # Clear gradients
        outputs = model(inputs)  # Forward pass
        loss = criterion(outputs, labels)  # Compute loss
        loss.backward()  # Backpropagation
        optimizer.step()  # Update weights

        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader):.4f}")

Epoch 1/10, Loss: 0.7448
Epoch 2/10, Loss: 0.7437
Epoch 3/10, Loss: 0.7437
Epoch 4/10, Loss: 0.7437
